In [94]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
import folium
from folium.plugins import HeatMap, MeasureControl, Fullscreen
import networkx as nx

In [95]:
crime_data = pd.read_csv("crime_data1.csv")

In [96]:

# Step 2: Graph Construction
# Create an empty graph
crime_graph = nx.Graph()

# Add nodes to the graph (crime locations)
crime_locations = [(row['Latitude'], row['Longitude']) for _, row in crime_data.iterrows()]
crime_graph.add_nodes_from(crime_locations)

# Define edges based on spatial proximity
# For example, connect nodes if their distance is below a certain threshold
threshold_distance = 0.1  # Adjust as needed
for i in range(len(crime_locations)):
    for j in range(i+1, len(crime_locations)):
        dist = np.linalg.norm(np.array(crime_locations[i]) - np.array(crime_locations[j]))
        if dist < threshold_distance:
            crime_graph.add_edge(crime_locations[i], crime_locations[j])


In [97]:
# Step 3: Feature Engineering

# Extract features for each node (crime location)
# Let's consider latitude, longitude, and crime type as features

# Assuming crime_data is a DataFrame containing crime data with columns: Crime, Latitude, Longitude
crime_locations = [(row['Latitude'], row['Longitude']) for _, row in crime_data.iterrows()]
crime_types = list(crime_data['Crime'])
node_features = np.array([[lat, lon, crime_type] for (lat, lon), crime_type in zip(crime_locations, crime_types)])

# Create a graph to compute additional features
crime_graph = nx.Graph()

# Add nodes to the graph (crime locations)
crime_graph.add_nodes_from(crime_locations)

# Define edges based on spatial proximity
# For example, connect nodes if their distance is below a certain threshold
threshold = 0.1  # Set your threshold distance here

# Function to compute distance between two points (latitude, longitude)
def compute_distance(point1, point2):
    lat1, lon1 = point1
    lat2, lon2 = point2
    # Use Haversine formula to compute distance
    # Implementation depends on your preference or available libraries
    # For simplicity, let's assume the distance is the Euclidean distance
    return np.sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)

# Add edges to the graph based on spatial proximity
for i in range(len(crime_locations)):
    for j in range(i + 1, len(crime_locations)):
        dist = compute_distance(crime_locations[i], crime_locations[j])
        if dist < threshold:
            crime_graph.add_edge(crime_locations[i], crime_locations[j])

# Now, crime_graph contains the graph structure with crime locations as nodes and edges based on spatial proximity.

# Compute centrality measures
# Degree centrality
degree_centralities = nx.degree_centrality(crime_graph)
# Betweenness centrality
betweenness_centralities = nx.betweenness_centrality(crime_graph)
# Closeness centrality
closeness_centralities = nx.closeness_centrality(crime_graph)

# Now you have extracted features for each node, including crime type, and computed three centrality measures.
# You can proceed to the next steps such as model selection and training.


In [98]:
# Step 4: Graph Representation
# Convert the graph into PyTorch Geometric data format
node_features = torch.tensor(crime_data[['Latitude', 'Longitude']].values, dtype=torch.float)
edge_index = 1  # Define edge indices
target = 4  # Define target labels if available
graph_data = Data(x=node_features, edge_index=edge_index, y=target)

In [99]:
# Step 5: Model Selection
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(node_features.size(1), 16)
        self.conv2 = GCNConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [100]:
# Step 6: Model Training
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

def train(model, data_loader, optimizer, criterion, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for batch in data_loader:
            optimizer.zero_grad()
            output = model(batch)
            loss = criterion(output, batch.y)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

In [101]:
# Step 7: Data Preparation for Training
train_loader = DataLoader(graph_data, batch_size=64, shuffle=True)

In [102]:
# Step 8: Model Training
train(model, train_loader, optimizer, criterion)

KeyError: 0

In [ ]:
# Step 9: Visualization
predicted_crime_rates = model(graph_data).detach().numpy()



In [ ]:
# Create a map centered at the mean latitude and longitude
center_lat = crime_data['Latitude'].mean()
center_lon = crime_data['Longitude'].mean()
crime_map = folium.Map(location=[center_lat, center_lon], zoom_start=10)



In [ ]:
# Add HeatMap layer to the map using predicted crime rates
HeatMap(data[['Latitude', 'Longitude', predicted_crime_rates]]).add_to(crime_map)



In [ ]:
# Add MeasureControl, Fullscreen control, and layer control
measure_control = MeasureControl(primary_length_unit='centimeters')
crime_map.add_child(measure_control)
Fullscreen().add_to(crime_map)
folium.LayerControl().add_to(crime_map)

# Save the map as HTML
crime_map.save("crime_prediction_map.html")